First thing is to run pystone3.py on a laptop to benchmark

 Profiling CPU

In [13]:
%cd measuring/

/Users/andymaheshw/pycon2017_measure/measuring


In [15]:
# %load profile_me.py
# file profile_me.py

"""Example to be profiled.
"""
import sys
import time

if sys.version_info.major < 3:
    range = xrange


def fast():
    """Wait 0.001 seconds.
    """
    time.sleep(1e-3)


def slow():
    """Wait 0.1 seconds.
    """
    time.sleep(0.1)


def use_fast():
    """Call `fast` 100 times.
    """
    for _ in range(100):
        fast()


def use_slow():
    """Call `slow` 100 times.
    """
    for _ in range(100):
        slow()


if __name__ == '__main__':
    use_fast()
    use_slow()


In [16]:
import cProfile

In [17]:
profile = cProfile.Profile()

In [19]:
profile.runcall(use_fast)

In [20]:
profile.print_stats()

         202 function calls in 0.130 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    0.001    0.000    0.130    0.001 <ipython-input-15-31b8b4df29c5>:13(fast)
        1    0.000    0.000    0.130    0.130 <ipython-input-15-31b8b4df29c5>:25(use_fast)
      100    0.129    0.001    0.129    0.001 {built-in method time.sleep}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [21]:
cProfile.run('use_fast()', 'fast.stats')

In [24]:
%ls *.stats

fast.stats


In [25]:
import pstats

In [26]:
stats = pstats.Stats('fast.stats')

In [32]:
stats.print_stats()

Thu May 18 09:44:46 2017    fast.stats

         204 function calls in 0.132 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.132    0.132 <string>:1(<module>)
      100    0.131    0.001    0.131    0.001 {built-in method time.sleep}
        1    0.000    0.000    0.132    0.132 <ipython-input-15-31b8b4df29c5>:25(use_fast)
      100    0.001    0.000    0.131    0.001 <ipython-input-15-31b8b4df29c5>:13(fast)
        1    0.000    0.000    0.132    0.132 {built-in method builtins.exec}




In [33]:
stats.print_callers()

   Random listing order was used

Function                                          was called by...
                                                      ncalls  tottime  cumtime
{method 'disable' of '_lsprof.Profiler' objects}  <- 
<string>:1(<module>)                              <-       1    0.000    0.132  {built-in method builtins.exec}
{built-in method time.sleep}                      <-     100    0.131    0.131  <ipython-input-15-31b8b4df29c5>:13(fast)
<ipython-input-15-31b8b4df29c5>:25(use_fast)      <-       1    0.000    0.132  <string>:1(<module>)
<ipython-input-15-31b8b4df29c5>:13(fast)          <-     100    0.001    0.131  <ipython-input-15-31b8b4df29c5>:25(use_fast)
{built-in method builtins.exec}                   <- 




In [34]:
stats.print_callees('use_fast')

   Random listing order was used
   List reduced from 6 to 1 due to restriction <'use_fast'>

Function                                      called...
                                                  ncalls  tottime  cumtime
<ipython-input-15-31b8b4df29c5>:25(use_fast)  ->     100    0.001    0.131  <ipython-input-15-31b8b4df29c5>:13(fast)




In [36]:
%prun use_fast()

In [38]:
%prun?

In [40]:
# %load clock_check.py
# file: measuring/clock_check.py

"""Checking different timing functions.
"""

from __future__ import print_function

import os
import sys
import time
import timeit


if sys.version_info.major < 3:
    range = xrange


def clock_check(duration=1):
    """Check the measured time with different methods.
    """
    start_os_time0 = os.times()[0]
    start_time_clock = time.clock()
    start_default_timer = timeit.default_timer()
    for _ in range(int(1e6)):
        1 + 1
    time.sleep(duration)
    durtation_os_time0 = os.times()[0] - start_os_time0
    durtation_time_clock = time.clock() - start_time_clock
    durtation_default_timer = timeit.default_timer() - start_default_timer
    print('durtation_os_time0:     ', durtation_os_time0)
    print('durtation_time_clock:   ', durtation_time_clock)
    print('durtation_default_timer:', durtation_default_timer)


if __name__ == '__main__':
    clock_check()


durtation_os_time0:      0.030000000000000027
durtation_time_clock:    0.03408499999999992
durtation_default_timer: 1.038308375995257


In [41]:
timeit.default_timer

<function time.perf_counter>

In [42]:
%timeit 1+1 

10000000 loops, best of 3: 20.5 ns per loop


In [43]:
import time

In [45]:
%%timeit
## should be a millionth of second a million times = 1 second (really 6 seconds)
for _ in range(10**6):
    time.sleep(1e-6)

1 loop, best of 3: 6.08 s per loop


In [48]:
%cd ../pi

/Users/andymaheshw/pycon2017_measure/pi


In [50]:
# %load simple_pi.py
# file: simple_pi.py

"""Calculating pi with Monte Carlo.
"""

from __future__ import print_function

import math
import random
import sys


if sys.version_info[0] < 3:
    range = xrange


def pi_plain(total):
    """Calculate pi with `total` hits.
    """
    count_inside = 0
    for _ in range(total):
        x = random.random()
        y = random.random()
        dist = math.sqrt(x * x + y * y)
        if dist < 1:
            count_inside += 1
    return 4.0 * count_inside / total

if __name__ == '__main__':

    def test():
        """Check if it works.
        """
        n = int(1e6)
        print('pi:', pi_plain(n))

    test()


pi: 3.139536


In [53]:
# %load numpy_pi.py
# file: numpy_pi.py
"""Calculating pi with Monte Carlo Method and NumPy.
"""

from __future__ import print_function

import numpy                                                   #1


def pi_numpy(total):                                           #2
    """Compute pi.
    """
    x = numpy.random.rand(total)                               #3
    y = numpy.random.rand(total)                               #4
    dist = numpy.sqrt(x * x + y * y)                           #5
    count_inside = len(dist[dist < 1])                         #6
    return 4.0 * count_inside / total

if __name__ == '__main__':

    def test():
        """Time the execution.
        """
        import timeit
        start = timeit.default_timer()
        pi_numpy(int(1e6))
        print('run time', timeit.default_timer() - start)
    test()


run time 0.056187758993473835


In [54]:
%load_ext snakeviz

In [57]:
n = 100000

In [58]:
%snakeviz pi_numpy(n)

 
*** Profile stats marshalled to file '/var/folders/gf/3tl_fl216b33yx72x1s83mkw0000gp/T/tmp08irqmet'. 


In [59]:
cd ../measuring/

/Users/andymaheshw/pycon2017_measure/measuring


In [60]:
pwd

'/Users/andymaheshw/pycon2017_measure/measuring'

In [62]:
% kernprof -v -l profile_me_use_line_profiler.py

ERROR: Line magic function `%kernprof` not found.


In [ ]:
# %load accumulate.py
# file accumulate.py

"""Simple test function for line_profiler.
"""

@profile
def accumulate(iterable):
    """Accumulate the intermediate steps in summing all elements.

    The result is a list with the length of `iterable`.
    The last element is the sum of all elements of `iterable`
    >>> accumulate(range(5))
    [0, 1, 3, 6, 10]
    >>> accumulate(range(10))
    [0, 1, 3, 6, 10, 15, 21, 28, 36, 45]
    """
    acm = [iterable[0]]
    for elem in iterable[1:]:
        old_value = acm[-1]
        new_value = old_value + elem
        acm.append(new_value)
    return acm


if __name__ == '__main__':
    accumulate(range(10))
    accumulate(range(100))


In [65]:
```
Total time: 0.000164 s
File: accumulate.py
Function: accumulate at line 6

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     6                                           @profile
     7                                           def accumulate(iterable):
     8                                               """Accumulate the intermediate steps in summing all elements.
     9
    10                                               The result is a list with the length of `iterable`.
    11                                               The last element is the sum of all elements of `iterable`
    12                                               >>> accumulate(range(5))
    13                                               [0, 1, 3, 6, 10]
    14                                               >>> accumulate(range(10))
    15                                               [0, 1, 3, 6, 10, 15, 21, 28, 36, 45]
    16                                               """
    17         2            5      2.5      3.0      acm = [iterable[0]]
    18       110           41      0.4     25.0      for elem in iterable[1:]:
    19       108           40      0.4     24.4          old_value = acm[-1]
    20       108           33      0.3     20.1          new_value = old_value + elem
    21       108           45      0.4     27.4          acm.append(new_value)
    22         2            0      0.0      0.0      return acm
```

SyntaxError: invalid syntax (<ipython-input-65-001e0cf60262>, line 1)

In [ ]:
File: calc.py
Function: calc at line 13

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    13                                           @profile
    14                                           def calc(number, loops=1000):
    15                                               """Do some math calculations.
    16                                               """
    17         1            8      8.0      0.0      sqrt = math.sqrt
    18    100001        34109      0.3      8.1      for x in range(loops):
    19    100000        37120      0.4      8.8          x = number + 10
    20    100000        37188      0.4      8.8          x = number * 10
    21    100000        67613      0.7     16.1          x = number ** 10
    22    100000        77917      0.8     18.5          x = pow(number, 10)
    23    100000        49913      0.5     11.9          x = math.sqrt(number)
    24    100000        43419      0.4     10.3          x = sqrt(number)
    25    100000        39918      0.4      9.5          math.sqrt
    26    100000        33404      0.3      7.9          sqrt

In [68]:
%load_ext line_profiler

In [71]:
%lprun -f use_fast -f fast use_fast()

In [74]:
# %load create_list.py
# file: create_list.py

import sys

if sys.version_info.major < 3:
    range = xrange


def insert_zero(n=int(1e4)):
    """Assemble list with `insert`. Inefficient.
    """
    L = []
    for x in range(n):
        L.insert(0, x)
    return L


def append_reverse(n=int(1e4)):
    """Assemble list with `append` and `reverse`.
    """
    L = []
    for x in range(n):
        L.append(x)
    L.reverse()
    return L


In [79]:
%lprun?

In [82]:
%lprun -f insert_zero insert_zero()